Task=Web Scraping

Website=www.zillow.com

By Laiba Meer

#Importing Necessary Libraries

In [40]:
import requests
from bs4 import BeautifulSoup as soup
#Set Header
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',

  'referer': 'https://www.zillow.com/homes/for_rent/Manhattan,-New-York,-NY_rb/?searchQueryState=%7B%22pagination'

}


In [41]:
#Send a get request:
url = 'https://www.zillow.com/homes/for_rent/Manhattan,-New-York,-NY_rb'
html = requests.get(url=url,headers=header)
html.status_code

200

In [ ]:
bsobj=soup(html.content,'lxml')
bsobj

#Listing prices

In [49]:
price_list = []
for price in bsobj.findAll('div',{'class':'StyledPropertyCardDataArea-c11n-8-84-3__sc-yipmu-0 dbDWjx'}):
  #print('price is: ', price.text.replace('bd','b|').replace('|s','|').replace('io','io|').strip().split('|')[:-1])
  price_list.append(price.text.replace('bd','b|').replace('|s','|').replace('o','o|').strip().split('|')[:-1])
price_list



[['$5,344+ 1 b', '$11,291+ 2 b'],
 ['$4,560+ 1 b'],
 ['$3,939+ 1 b', '$5,092+ 2 b', '$6,324+ 3 b'],
 ['$6,866+ 2 b'],
 ['$4,950+ 1 b', '$8,585+ 2 b'],
 ['$3,883+ 1 b', '$6,288+ 2 b'],
 ['$5,769+ 1 b'],
 ['$5,380+ 1 b', '$8,310+ 2 b'],
 ['$8,495+ 2 b']]

#Listing address

In [100]:
address = []
for adr in bsobj.findAll('div',{'class':'StyledPropertyCardDataWrapper-c11n-8-84-3__sc-1omp4c3-0 bKpguY property-card-data'}):
  address.append(adr.a.text.strip())
address

['19 Dutch | 19 Dutch St, New York, NY',
 'AVA High Line | 525 W 28th St, New York, NY',
 'The Oriana | 420 E 54th St, New York, NY',
 'The Chelsea | 160 W 24th St, New York, NY',
 'The Eugene | 435 W 31st St, New York, NY',
 'Avalon Midtown West | 250 W 50th St, New York, NY',
 'The Dylan | 309 5th Ave, New York, NY',
 'Avalon Bowery Place | 11 E 1st St, New York, NY',
 'Two Lincoln Square | 60 W 66th St, New York, NY']

In [108]:
# Assuming 'Titles' is the list you obtained
titles = [addr.split('|')[0].strip() for addr in address]

print(titles)


['19 Dutch', 'AVA High Line', 'The Oriana', 'The Chelsea', 'The Eugene', 'Avalon Midtown West', 'The Dylan', 'Avalon Bowery Place', 'Two Lincoln Square']


In [114]:
# Assuming 'address' is the list you obtained
adress = [addr.split('|')[1].strip() for addr in address]

print(adress)

['19 Dutch St, New York, NY', '525 W 28th St, New York, NY', '420 E 54th St, New York, NY', '160 W 24th St, New York, NY', '435 W 31st St, New York, NY', '250 W 50th St, New York, NY', '309 5th Ave, New York, NY', '11 E 1st St, New York, NY', '60 W 66th St, New York, NY']


#URL Of listing

In [144]:
website_links = []

# Assuming the base URL of the website
base_url = 'https://www.zillow.com'  # Replace with the actual base URL

for adr in bsobj.findAll('div', {'class': 'StyledPropertyCardDataWrapper-c11n-8-84-3__sc-1omp4c3-0 bKpguY property-card-data'}):
    # Assuming the website link is within a child <a> tag, adjust this based on the HTML structure
    relative_url = adr.find('a', {'class': 'property-card-link'})['href']

    # Construct the absolute URL
    absolute_url = base_url + relative_url

    website_links.append(absolute_url)

print(website_links)

['https://www.zillow.com/apartments/manhattan-ny/19-dutch/B3qnH3/', 'https://www.zillow.com/apartments/new-york-ny/ava-high-line/4MH7/', 'https://www.zillow.com/apartments/manhattan-ny/the-oriana/5XWNCh/', 'https://www.zillow.com/apartments/new-york-ny/the-chelsea/5XT2qW/', 'https://www.zillow.com/apartments/new-york-ny/the-eugene/B73Y66/', 'https://www.zillow.com/apartments/new-york-ny/avalon-midtown-west/5XVMHG/', 'https://www.zillow.com/apartments/manhattan-ny/the-dylan/3LSS/', 'https://www.zillow.com/apartments/new-york-ny/avalon-bowery-place/5XVHKv/', 'https://www.zillow.com/apartments/manhattan-ny/two-lincoln-square/9gpqwm/']


#Dataframe

In [145]:
import pandas as pd
df = pd.DataFrame(price_list,columns=['Price1','Price2','Price3'])
df['Listing_Title'] = titles
df['Address'] = adress
df['URL']=website_links


In [146]:
# Rearrange the columns
df = df[['Listing_Title', 'Address', 'Price1', 'Price2', 'Price3','URL']]

In [147]:
df

,Listing_Title,Address,Price1,Price2,Price3,URL
0,19 Dutch,"19 Dutch St, New York, NY","$5,344+ 1 b","$11,291+ 2 b",None,https://www.zillow.com/apartments/manhattan-ny...
1,AVA High Line,"525 W 28th St, New York, NY","$4,560+ 1 b",None,None,https://www.zillow.com/apartments/new-york-ny/...
2,The Oriana,"420 E 54th St, New York, NY","$3,939+ 1 b","$5,092+ 2 b","$6,324+ 3 b",https://www.zillow.com/apartments/manhattan-ny...
3,The Chelsea,"160 W 24th St, New York, NY","$6,866+ 2 b",None,None,https://www.zillow.com/apartments/new-york-ny/...
4,The Eugene,"435 W 31st St, New York, NY","$4,950+ 1 b","$8,585+ 2 b",None,https://www.zillow.com/apartments/new-york-ny/...
5,Avalon Midtown West,"250 W 50th St, New York, NY","$3,883+ 1 b","$6,288+ 2 b",None,https://www.zillow.com/apartments/new-york-ny/...
6,The Dylan,"309 5th Ave, New York, NY","$5,769+ 1 b",None,None,https://www.zillow.com/apartments/manhattan-ny...
7,Avalon Bowery Place,"11 E 1st St, New York, NY","$5,380+ 1 b","$8,310+ 2 b",None,https://www.zillow.com/apartments/new-york-ny/...
8,Two Lincoln Square,"60 W 66th St, New York, NY","$8,495+ 2 b",None,None,https://www.zillow.com/apartments/manhattan-ny...


In [148]:
df.to_csv('zillow_dataframe.csv', index=False)